In [1]:
import numpy as np
from skvideo.io import FFmpegReader, ffprobe
from skvideo.utils import rgb2gray
from PIL import Image
from keras.preprocessing import image
from tqdm import tqdm
import matplotlib.pyplot as plt


class Videos(object):

    def __init__(self, target_size=None, to_gray=True, max_frames=None,
                 extract_frames='middle', required_fps=None,
                 normalize_pixels=None):
        """
        Initializing the config variables
        Parameters:
            target_size (tuple): (New_Width, New_Height), Default 'None'
                A tuple denoting the target width and height of each frame in each of the video
            to_gray (boolean): Default 'True'
                If True, then each frame will be converted to gray scale. Otherwise, not.
            max_frames (int): Default 'None'
                The maximum number of frames to return for each video.
                Extra frames are removed based on the value of 'extract_frames'.
            extract_frames (str): {'first', 'middle', 'last'}, Default 'middle'
                'first': Extract the first 'N' frames
                'last': Extract the last 'N' frames
                'middle': Extract 'N' frames from the middle
                    Remove ((total_frames - max_frames) // 2) frames from the beginning as well as the end
            required_fps (int): Default 'None'
                Capture 'N' frame(s) per second from the video.
                Only the first 'N' frame(s) for each second in the video are captured.
            normalize_pixels (tuple/str): Default 'None'
                If 'None', the pixels will not be normalized.
                If a tuple - (New_min, New_max) is passed, Min-max Normalization will be used.
                If the value is 'z-score', then Z-score Normalization will be used.
                For each pixel p, z_score = (p - mean) / std
        """

        self.target_size = target_size
        self.to_gray = to_gray
        self.max_frames = max_frames
        self.extract_frames = extract_frames
        self.required_fps = required_fps
        self.normalize_pixels = normalize_pixels
        self.fps = None

    def read_videos(self, paths):
        """
        Parameters:
            paths (list): Required
                 A list of paths of the videos to be read
        Returns:
            Numpy.ndarray
                A 5-d tensor with shape (<No. of Videos>, <No. of frames>, <height>, <width>, <channels>)
        """

        list_of_videos = [
            self._read_video(path) for path in tqdm(paths)
        ]
        for video in list_of_videos:
            print(video.shape)            
        tensor = np.vstack(list_of_videos)

        if self.normalize_pixels != None:
            # Pixels are normalized for each video individually
            if (type(self.normalize_pixels) == tuple) and (len(self.normalize_pixels) == 2):
                base = self.normalize_pixels[0]
                r = self.normalize_pixels[1] - base
                min_ = np.min(tensor, axis=(1, 2, 3), keepdims=True)
                max_ = np.max(tensor, axis=(1, 2, 3), keepdims=True)
                return ((tensor.astype('float32') - min_) / (max_ - min_)) * r + base

            elif self.normalize_pixels == 'z-score':
                mean = np.mean(tensor, axis=(1, 2, 3), keepdims=True)
                std = np.std(tensor, axis=(1, 2, 3), keepdims=True)
                return (tensor.astype('float32') - mean) / std
            
            else:
                raise ValueError('Invalid value of \'normalize_pixels\'')

        return tensor

    def get_frame_count(self, paths):
        """
        Can be used to determine the value of `max_frames`
        Parameters:
            paths (list): Required
                 A list of paths of the videos to be read
        Returns:
            dict (python dictionary)
                For each video, the total number of frames in that video is stored in the dictionary.
        """

        frame_count = {}
        for path in paths:
            cap = FFmpegReader(filename=path)
            frame_count[path] = cap.inputframenum
            cap.close()

        return frame_count

    def _read_video(self, path):
        """
        Parameters:
            path (str): Required
                Path of the video to be read
        Returns:
            Numpy.ndarray
                A 5-d tensor with shape (1, <No. of frames>, <height>, <width>, <channels>)
        """

        cap = FFmpegReader(filename=path)
        list_of_frames = []
        self.fps = int(cap.inputfps)                  # Frame Rate

        for index, frame in enumerate(cap.nextFrame()):

            capture_frame = True
            if self.required_fps != None:
                is_valid = range(self.required_fps)
                capture_frame = (index % self.fps) in is_valid

            if capture_frame:

                if self.target_size is not None:
                    temp_image = image.array_to_img(frame)
                    frame = image.img_to_array(
                        temp_image.resize(
                            self.target_size,
                            Image.ANTIALIAS)).astype('uint8')

                # Shape of each frame -> (<height>, <width>, 3)
                list_of_frames.append(frame)

        temp_video = np.stack(list_of_frames)
        cap.close()

        if self.to_gray:
            temp_video = rgb2gray(temp_video)
                
        if self.max_frames is not None:
            temp_video = self._process_video(video=temp_video,path=path)            
        print(np.expand_dims(temp_video, axis=0).shape)
        print(path)
        print("")
        return np.expand_dims(temp_video, axis=0)

    def _process_video(self, video,path):
        """
        Parameters:
            video (Numpy.ndarray):
                Shape = (<No. of frames>, <height>, <width>, <channels>)
                Video whose frames are to be extracted
        Returns:
            Numpy.ndarray
                A tensor (processed video) with shape (<`max_frames`>, <height>, <width>, <channels>)
        """        
        
        total_frames = video.shape[0]
        if self.max_frames <= total_frames:
            if self.extract_frames == 'first':
                video = video[:self.max_frames]
            elif self.extract_frames == 'last':
                video = video[(total_frames - self.max_frames):]
            elif self.extract_frames == 'middle':
                # No. of frames to remove from the front
                front = ((total_frames - self.max_frames) // 2) + 1
                video = video[front:(front + self.max_frames)]            
            else:
                raise ValueError('Invalid value of \'extract_frames\'')                
        else:
            raise ValueError('Invalid number of frames')                
        return video

Using TensorFlow backend.


In [15]:
# Imports
import os
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

# Loading the data
raw_data = load_files(r"C:\Users\gural\Desktop\animals", shuffle=False)
files = raw_data['filenames']
targets = raw_data['target']

# Randomly dividing the whole data into training (66.67%) and testing (33.33%) data 
train_files, test_files, train_targets, test_targets = train_test_split(files, targets, test_size=1/6, random_state=191)

# Taking ~25% of the training data for validation
valid_files = train_files[500:]
valid_targets = train_targets[500:]

# Remaining data will be used for training the model
train_files = train_files[:500]
train_targets = train_targets[:500]

# Generic details about the data
print('Total number of videos:', len(files))
print('\nNumber of videos in training data:', train_files.shape[0])
print('Number of videos in validation data:', valid_files.shape[0])
print('Number of videos in test data:', test_files.shape[0])

Total number of videos: 746

Number of videos in training data: 500
Number of videos in validation data: 121
Number of videos in test data: 125


In [16]:
print('The categorical labels are converted into integers.\nFollowing is the mapping - \n')
for label in zip(range(16), raw_data['target_names']):
    print(label)

The categorical labels are converted into integers.
Following is the mapping - 

(0, 'cut')
(1, 'fly')
(2, 'sleep')
(3, 'stand')


In [17]:
# Displaying the first 5 videos (paths) in the training data along with their labels
# (path of video, class label)
for pair in zip(train_files[:16], train_targets[:16]):
    print(pair)

('C:\\Users\\gural\\Desktop\\animals\\cut\\VID_20190515_184703Rotate.mp4', 0)
('C:\\Users\\gural\\Desktop\\animals\\fly\\HorizontalFlipVID_20190327_190039Rotate.mp4', 1)
('C:\\Users\\gural\\Desktop\\animals\\stand\\HorizontalFlipVID_20190327_174806translate.mp4', 3)
('C:\\Users\\gural\\Desktop\\animals\\stand\\VID_20190515_185152Crop.mp4', 3)
('C:\\Users\\gural\\Desktop\\animals\\sleep\\HorizontalFlipVID_20190327_190359Rotate40.mp4', 2)
('C:\\Users\\gural\\Desktop\\animals\\cut\\HorizontalFlipVID_20190327_190427Crop.mp4', 0)
('C:\\Users\\gural\\Desktop\\animals\\stand\\HorizontalFlipVID_20190327_185715.mp4', 3)
('C:\\Users\\gural\\Desktop\\animals\\fly\\HorizontalFlipVID_20190515_184843Rotate.mp4', 1)
('C:\\Users\\gural\\Desktop\\animals\\stand\\HorizontalFlipVID_20190515_185146Rotate40.mp4', 3)
('C:\\Users\\gural\\Desktop\\animals\\stand\\HorizontalFlipVID_20190327_185946.mp4', 3)
('C:\\Users\\gural\\Desktop\\animals\\fly\\VID_20190327_190205Rotate40.mp4', 1)
('C:\\Users\\gural\\Deskt

In [18]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# The path of a sample video in the training data
sample_files = train_files[1:2]

# An object of the class 'Videos'
reader = Videos(target_size=None, 
                to_gray=False)
print(sample_files)
# Loading the sample videos, in their original format
sample = reader.read_videos(sample_files)
print('\nShape of the sample data:', sample.shape)

['C:\\Users\\gural\\Desktop\\animals\\fly\\HorizontalFlipVID_20190327_190039Rotate.mp4']


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

(1, 30, 1080, 1920, 3)
C:\Users\gural\Desktop\animals\fly\HorizontalFlipVID_20190327_190039Rotate.mp4



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


(1, 30, 1080, 1920, 3)

Shape of the sample data: (1, 30, 1080, 1920, 3)


In [19]:
import numpy as np
from keras.utils import to_categorical

#num of frames in alon phone and computer are 40 per second
reader = Videos(target_size=(128, 128), 
                to_gray=True, 
                max_frames=23, 
                extract_frames='first', 
                normalize_pixels=(0, 1))

In [20]:
# Reading training videos and one-hot encoding the training labels
X_train = reader.read_videos(train_files)
y_train = to_categorical(train_targets, num_classes=4)
print('Shape of training data:', X_train.shape)
print('Shape of training labels:', y_train.shape)

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\cut\VID_20190515_184703Rotate.mp4



  0%|▏                                                                                 | 1/500 [00:03<29:53,  3.59s/it]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\fly\HorizontalFlipVID_20190327_190039Rotate.mp4



  0%|▎                                                                                 | 2/500 [00:07<29:53,  3.60s/it]

...

100%|███████████████████████████████████████████████████████████████████████████████▊| 499/500 [41:13<00:07,  7.23s/it]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\sleep\HorizontalFlipVID_20190327_190259Rotate.mp4



100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [41:19<00:00,  6.78s/it]


(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 

(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 

In [21]:
# Reading validation videos and one-hot encoding the validation labels
X_valid = reader.read_videos(valid_files)
y_valid = to_categorical(valid_targets, num_classes=4)
print('Shape of validation data:', X_valid.shape)
print('Shape of validation labels:', y_valid.shape)

  0%|                                                                                          | 0/121 [00:00<?, ?it/s]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\stand\HorizontalFlipVID_20190327_185710Rotate40.mp4



  1%|▋                                                                                 | 1/121 [00:07<15:26,  7.72s/it]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\fly\VID_20190327_190045translate.mp4



  2%|█▎                                                                                | 2/121 [00:14<14:32,  7.33s/it]

...

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\cut\VID_20190327_190440translate.mp4



 99%|███████████████████████████████████████████████████████████████████████████████▎| 120/121 [13:11<00:06,  6.34s/it]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\cut\HorizontalFlipVID_20190327_175707Rotate40.mp4



100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [13:19<00:00,  6.59s/it]


(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 

In [ ]:
# Reading testing videos and one-hot encoding the testing labels
X_test = reader.read_videos(test_files)
y_test = to_categorical(test_targets, num_classes=4)
print('Shape of testing data:', X_test.shape)
print('Shape of testing labels:', y_test.shape)

  0%|                                                                                          | 0/125 [00:00<?, ?it/s]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\stand\HorizontalFlipVID_20190327_185941Rotate40.mp4



  1%|▋                                                                                 | 1/125 [00:06<14:09,  6.85s/it]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\cut\VID_20190327_190518Rotate40.mp4



  2%|█▎                                                                                | 2/125 [00:12<13:21,  6.52s/it]

...

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\cut\HorizontalFlipVID_20190327_190427translate.mp4



 99%|███████████████████████████████████████████████████████████████████████████████▎| 124/125 [14:03<00:06,  7.00s/it]

(1, 23, 128, 128, 1)
C:\Users\gural\Desktop\animals\fly\HorizontalFlipVID_20190515_184843Rotate40.mp4



100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [14:17<00:00,  9.14s/it]


(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 128, 1)
(1, 23, 128, 

In [ ]:
print(X_train.shape[1:])

(23, 128, 128, 1)


In [ ]:
# Imports
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D
from keras.layers.core import Dense,Dropout

# Using the Sequential Model
model = Sequential()

# Adding Alternate convolutional and pooling layers
model.add(Conv3D(filters=4, kernel_size=(3, 3, 3), strides=(5, 1, 1), padding='same', activation='relu', 
                 input_shape=X_train.shape[1:]))
model.add(Dense(128, activation='relu'))


model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
model.add(Conv3D(filters=128, kernel_size=(1, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
model.add(Conv3D(filters=256, kernel_size=(1, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
model.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))
model.add(Dense(128, activation='relu'))

model.add(Conv3D(filters=512, kernel_size=(1, 3, 3), strides=(3, 1, 1), padding='valid', activation='relu'))
model.add(MaxPooling3D(pool_size=2, strides=(1, 2, 2), padding='same'))

# A global average pooling layer to get a 1-d vector
# The vector will have a depth (same as number of elements in the vector) of 256
model.add(GlobalAveragePooling3D())

# The Global average pooling layer is followed by a fully-connected neural network, with one hidden and one output layer

# Hidden Layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))

# Output layer
model.add(Dense(4, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 5, 128, 128, 4)    112       
_________________________________________________________________
dense_6 (Dense)              (None, 5, 128, 128, 128)  640       
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 1, 126, 126, 64)   221248    
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 1, 124, 124, 128)  73856     
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 1, 122, 122, 256)  295168    
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 1, 61, 61, 256)    0         
_________________________________________________________________
dense_7 (Dense)              (None, 1, 61, 61, 128)    32896     
__________

In [ ]:
# Imports
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow
from keras.optimizers import Adam

class LossAccHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracy = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('acc'))
        
history = LossAccHistory()


# Compiling the model
model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr=0.0008,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=None,
    decay=0.0,
    amsgrad=False), metrics=["accuracy"])

# Saving the model that performed the best on the validation set
checkpoint = ModelCheckpoint(filepath='actions.best.hdf5', save_best_only=True, 
                             verbose=1,save_weights_only=False, mode='auto')

# Training the model for 40 epochs
history = model.fit(X_train, y_train, batch_size=100, epochs=60, 
                    validation_data=(X_valid, y_valid), verbose=2, callbacks = [history, checkpoint])

Train on 500 samples, validate on 121 samples
Epoch 1/60
 - 3306s - loss: 1.4043 - acc: 0.2560 - val_loss: 1.4009 - val_acc: 0.2479

Epoch 00001: val_loss improved from inf to 1.40088, saving model to actions.best.hdf5
Epoch 2/60
 - 2008s - loss: 1.3930 - acc: 0.2620 - val_loss: 1.4107 - val_acc: 0.2562

Epoch 00002: val_loss did not improve from 1.40088
Epoch 3/60
 - 2060s - loss: 1.4178 - acc: 0.2560 - val_loss: 1.3894 - val_acc: 0.2149

Epoch 00003: val_loss improved from 1.40088 to 1.38935, saving model to actions.best.hdf5
Epoch 4/60
 - 1927s - loss: 1.3930 - acc: 0.2500 - val_loss: 1.3861 - val_acc: 0.2810

Epoch 00004: val_loss improved from 1.38935 to 1.38609, saving model to actions.best.hdf5
Epoch 5/60


In [13]:
# Loading the model that performed the best on the validation set
model.load_weights('clothes.best.hdf5')

# Testing the model on the Test data
(loss, accuracy) = model.evaluate(X_test, y_test, batch_size=70, verbose=0)

print('Accuracy on test data: {:.2f}%'.format(accuracy * 100))

Accuracy on test data: 58.82%


In [17]:
from tensorflow.keras.models import load_model
# Imports
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow
from keras.optimizers import Adam


# Saving the model that performed the best on the validation set
checkpoint = ModelCheckpoint(filepath='Model_1.batchsize70.bestContinue.hdf5', save_best_only=True, 
                             verbose=1,save_weights_only=False, mode='auto')


model = load_model('Model_1.weights.best.hdf5')
# Training the model for 40 epochs
history = model.fit(X_train, y_train, batch_size=70, epochs=40, 
                    validation_data=(X_valid, y_valid), verbose=2, callbacks = [history, checkpoint])

Train on 525 samples, validate on 146 samples
Epoch 1/40

Epoch 00001: val_loss improved from inf to 1.45895, saving model to Model_1.batchsize70.bestContinue.hdf5
 - 2454s - loss: 1.6351 - acc: 0.2648 - val_loss: 1.4589 - val_acc: 0.2603
Epoch 2/40


KeyboardInterrupt: 